In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


2.0.0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
matplotlib 3.1.3
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


## 设置GPU

In [ ]:
tf.debugging.set_log_device_placement(True)   # 打印模型的GPU分布
gpus = tf.config.experimental.list_physical_devices('GPU')  # 获取所有的GPU
tf.config.experimental.set_visible_devices(gpus[-1], 'GPU') # 设置可见的GPU，其他没设置可见的GPU会变得不可能 
# 逻辑切分GPU，此时GPU不能设置自增长
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072),    # 将第一个GPU分成两个逻辑分区，内存都设为3072M
     tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072)]
)
print(len(gpus))
logical_gpu = tf.config.experimental.list_logical_devices('GPU')  # 获取所有的逻辑GPU
print(len(logical_gpu))

## 处理数据

### 读取数据

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)


### 生成dataset

In [ ]:
def make_dataset(images, labels, epochs, batsh_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)   # prefetch: 从训练集中预先取出多少个数据为训练做准备，用来加速
    return dataset

epochs = 100
batch_size = 128
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)

## 实现卷积神经网络模型

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32, 
                              kernel_size=3, 
                              padding='same', 
                              activation='relu', 
                              input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(filters=32, 
                              kernel_size=3, 
                              padding='same', 
                              activation='relu', 
                              input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=64,    # 在maxpooling后一般会将filters的值翻倍，以缓解maxpooling带来的参数损失 
                              kernel_size=3, 
                              padding='same', 
                              activation='relu'))
model.add(keras.layers.Conv2D(filters=64, 
                              kernel_size=3, 
                              padding='same', 
                              activation='relu', 
                              input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=128, 
                              kernel_size=3, 
                              padding='same', 
                              activation='relu', 
                              input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(filters=128, 
                              kernel_size=3, 
                              padding='same', 
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_dataset,
                    steps_per_epoch = x_train_scaled.shape[0] // batch_size,
                    epochs=10)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaled, y_test)